## CNN으로 MNIST 분류하기
이번 챕터에서는 CNN으로 MNIST를 분류해보겠습니다.

#### 1. 모델 이해하기
우리가 만들 모델의 아키텍처를 이해해봅시다. 합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위로 조금 다릅니다.

#### 1-1. 첫번째 표기방법
`합성곱(nn.Conv2d)` + `활성화 함수(nn.ReLU)`를 하나의 합성곱 층으로 보고, `맥스풀링(nn.MaxPool2d)`은 풀링 층으로 별도도 명명합니다.

#### 1-2. 두번째 표기방법
`합성곱(nn.Conv2d)` + `활성화 함수(nn.ReLU)` + `맥스풀링(nn.MaxPool2d)`을 하나의 합성곱 층으로 봅니다.

다시 말해 풀링도 하나의 층으로 보느냐, 안 보느냐의 문제인데 누가 옳고 틀리냐의 문제는 아니므로, 이번 챕터에서는 편의를 위해 맥스풀링까지도 포함하여 하나의 합성곱 층으로 판단하고 정리해보겠습니다. 다시 말해 두번째 표기 방법을 택하겠습니다.

모델의 아키텍처는 총 3개의 층으로 구성됩니다.

```py
# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
```

이를 직접 구현해보며 이해해봅시다

## 모델 구현하기
위의 3개의 층을 직접 구현해보겠습니다.

In [1]:
import torch
import torch.nn as nn


C:\Users\LUDOBICO\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


임의의 텐서를 만듭니다. 텐서의 크기는 1 x 1 x 28 x 28 입니다.

In [9]:
# 배치 크기 x 채널 x 높이 x 너비의 텐서를 선언
inputs = torch.Tensor(1,1,28,28)

print('텐서의 크기 {}'.format(inputs.shape))

텐서의 크기 torch.Size([1, 1, 28, 28])


#### 합성곱층과 풀링 선언하기
이제 첫번째 합성곱 층을 구현해봅시다. 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.

In [10]:
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이제 두번째 합성곱 층을 구현해봅시다. 32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.

In [11]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이제 맥스풀링을 구현해봅시다. 정수 하나를 인자로 넣으면 `커널 사이즈`와 `스트라이드` 둘 다 해당값으로 지정됩니다.

In [12]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 구현체를 연결하여 모델 만들기
지금까지는 선언만한 것이고 아직 이들을 연결시키지는 않았습니다. 이들을 연결시켜서 모델을 완성시켜보겠습니다. 우선 입력을 첫번째 합성곱층을 통과시키고 합성곱층을 통과시킨 후의 텐서의 크기를 보겠습니다.

In [13]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


`32채널`의 `28너비`,`28높이`의 텐서가 되었습니다. 32가 나온 이유는 `conv1`의 out_channel로 32를 지정해주었기 때문입니다. 또한, 28 너비 28 높이가 된 이유는 패딩을 1폭으로 하고 3 x 3 커널을 사용하면 크기가 보존되기 때문입니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다.

In [14]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


`32채널`의 `14너비`,`14높이`의 텐서가 되었습니다. 이제 이를 다시 두번째 합성곱층엥 통과시키고 통과한 후의 텐서의 크기를 보겠습니다.

In [15]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


`64채널`의 `14너비`,`14높이`의 텐서가 되었습니다. 64가 나온 이유는 `conv2`의 out_channel로 64를 지정해주었기 때문입니다. 또한, 14너비 14높이가 된 이유는 패딩을 1폭으로 하고 3 x 3 커널을 사용하면 크기가 보존되기 때문입니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다.

In [16]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이제 이 텐서를 펼치는 작업을 할 겁니다. 그런데 펼치기에 앞서 텐서의 n번째 차원을 접근하게 해주는 `.size(n)`에 대해서 배워보겠습니다. 현재 out의 크기는 1 x 64 x 7 x 7입니다. out의 첫번째 차원이 몇인지 출력해보겠습니다.

In [17]:
print('첫 번째 out사이즈: ', out.size(0))
print('두 번째 out사이즈: ', out.size(1))
print('세 번째 out사이즈: ', out.size(2))
print('네 번째 out사이즈: ', out.size(3))


첫 번째 out사이즈:  1
두 번째 out사이즈:  64
세 번째 out사이즈:  7
네 번째 out사이즈:  7


이제 이를 가지고 `.view()`를 사용하여 텐서를 펼치는 작업을 해보겠습니다.

In [18]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있습니다. 이제 이에 대해서 `전결합층(Fully-Connected Layer)`을 통과시켜보겠습니다. 출력층으로 10개의 뉴런을 배치하여 10개의 차원의 텐서로 변환합니다.

In [19]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## CNN으로 MNIST 분류하기
우선 필요한 도구들을 임포트합니다.

In [8]:
import torch
import torchvision.datasets
import torchvision.transforms
import torch.nn as nn
import torch.nn.init


만약 GPU를 사용 가능하다면 Device 값이 cuda가 되고, 아니라면 cpu가 됩니다.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.manual_seed(777)

print(device)

cuda


In [4]:
learning_rate = 1e-3
training_epochs = 15
batch_size = 100

데이터로더를 사용하여 데이터를 다루기 위해서 데이터셋을 정의해줍니다.

In [5]:
mnist_train = torchvision.datasets.MNIST(root='MNIST_DATA/',train=True, transform=torchvision.transforms.ToTensor(), download=True)

mnist_test = torchvision.datasets.MNIST(root='MNIST_DATA/',train=False, transform=torchvision.transforms.ToTensor(), download=True)

100.0%


Extracting MNIST_DATA/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_DATA/MNIST\raw



100.0%

Extracting MNIST_DATA/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_DATA/MNIST\raw




100.0%
100.0%

Extracting MNIST_DATA/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_DATA/MNIST\raw

Extracting MNIST_DATA/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_DATA/MNIST\raw



데이터로더를 사용하여 배치 크기를 지정해줍니다. 만약 데이터셋과 데이터로더가 기억이 안 난다면 `미니 배치와 데이터 로드` 챕터를 꼭 복습하세요.

In [7]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

이제 클래스로 모델을 설계합니다.

In [9]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # IMGinput shape=(?, 28, 28, 1)
        # conv (?, 28, 28, 32)
        # pool (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        #두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

모델을 정의합니다.

In [10]:
# CNN 모델 정의
model = CNN().to(device)

비용 함수와 옵티마이저를 정의합니다.

In [19]:
# criterion = nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수 포함되어져 있음
# criterion = nn.functional.cross_entropy()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 배치의 수를 출력해보겠습니다.

In [20]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


총 배치의 수는 600입니다. 그런데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개란 의미입니다. 이제 모델을 훈련시켜보겠습니다.

In [23]:
for epoch in range(training_epochs):
    avg_cost = 0

    for x, y in data_loader: #미니 배치 단위로 꺼내온다. x는 미니배치, y는 레이블
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        H = model(x)
        cost = criterion = nn.functional.cross_entropy(H,y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.234457567
[Epoch:    2] cost = 0.0695326701
[Epoch:    3] cost = 0.0514512435
[Epoch:    4] cost = 0.0416315421
[Epoch:    5] cost = 0.0356964469
[Epoch:    6] cost = 0.0288178064
[Epoch:    7] cost = 0.0248047356
[Epoch:    8] cost = 0.0211930908
[Epoch:    9] cost = 0.0175663717
[Epoch:   10] cost = 0.0157037005
[Epoch:   11] cost = 0.0130460002
[Epoch:   12] cost = 0.0119220605
[Epoch:   13] cost = 0.00998420455
[Epoch:   14] cost = 0.00845041033
[Epoch:   15] cost = 0.00690764049


이제 테스트를 해보겠습니다.

In [24]:
# 이제 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device=device)
    y_test = mnist_test.test_labels.to(device=device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())


c:\Users\aqs45\OneDrive\바탕 화면\repo\Python_AI\venv\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\aqs45\OneDrive\바탕 화면\repo\Python_AI\venv\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9824999570846558
